In [ ]:
pip install git+https://www.github.com/keras-team/keras-contrib.git

In [ ]:
# example of preparing the horses and zebra dataset
from os import listdir
from numpy import asarray
from numpy import vstack
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from numpy import savez_compressed
from random import random
from numpy import load
from numpy import zeros
from numpy import ones
from numpy import asarray
from numpy.random import randint
from tensorflow.keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import Model, load_model
from keras.models import Input
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from matplotlib import pyplot

In [ ]:
pip install scikit-image

In [ ]:
# example of calculating the frechet inception distance in Keras
import numpy
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy import asarray
from numpy.random import randint
from scipy.linalg import sqrtm
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.datasets.mnist import load_data
from skimage.transform import resize
 
# scale an array of images to a new size
def scale_images(images, new_shape):
    images_list = list()
    for image in images:
    # resize with nearest neighbor interpolation
        new_image = resize(image, new_shape, 0)
        # store
        images_list.append(new_image)
    return asarray(images_list)
 
# calculate frechet inception distance
def calculate_fid(model, images1, images2):
 # calculate activations
    act1 = model.predict(images1)
    act2 = model.predict(images2)
    print('Inceptionv3 prediction done')
    # calculate mean and covariance statistics
    mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
    mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
    # calculate sum squared difference between means
    ssdiff = numpy.sum((mu1 - mu2)**2.0)
    # calculate sqrt of product between cov
    covmean = sqrtm(sigma1.dot(sigma2))
    # check and correct imaginary numbers from sqrt
    if iscomplexobj(covmean):
        covmean = covmean.real
    # calculate score
    fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid
 
# prepare the inception v3 model
model = InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))
# # define two fake collections of images
# images1 = randint(0, 255, 10*32*32*3)
# images1 = images1.reshape((10,32,32,3))
# images2 = randint(0, 255, 10*32*32*3)
# images2 = images2.reshape((10,32,32,3))
# print('Prepared', images1.shape, images2.shape)
# # convert integer to floating point values
# images1 = images1.astype('float32')
# images2 = images2.astype('float32')
def FID(images1,images2):    
    # resize images
    images1 = scale_images(images1, (299,299,3))
    images2 = scale_images(images2, (299,299,3))
    print('Scaled', images1.shape, images2.shape)
    # pre-process images
    images1 = preprocess_input(images1)
    images2 = preprocess_input(images2)
    print('Preprocessed_Images')
#    print('Calculating FID')
    fid = calculate_fid(model, images1, images2)
    print('FID (different): %.3f' % fid)
    return fid

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

# define the discriminator model
def define_discriminator(image_shape):
	# weight initialization
	init = RandomNormal(stddev=0.02, seed = 20)
	# source image input convoluted sie = (w-f+2p)/s + 1
	in_image = Input(shape=image_shape)
	# C64 - (256-4+2*2)/2 + 1 = 128+1
	d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(in_image)
	d = LeakyReLU(alpha=0.2)(d)
	# C128 
	d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = InstanceNormalization(axis=-1)(d)
	d = LeakyReLU(alpha=0.2)(d)
	# C256
	d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = InstanceNormalization(axis=-1)(d)
	d = LeakyReLU(alpha=0.2)(d)
	# C512
	d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = InstanceNormalization(axis=-1)(d)
	d = LeakyReLU(alpha=0.2)(d)
	# second last output layer
	d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
	d = InstanceNormalization(axis=-1)(d)
	d = LeakyReLU(alpha=0.2)(d)
	# patch output
	patch_out = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
	# define model
	model = Model(in_image, patch_out)
	# compile model
	model.compile(loss='mse', optimizer=Adam(lr=0.000005, beta_1=0.5), loss_weights=[0.5])
	return model
 
# generator a resnet block
def resnet_block(n_filters, input_layer):
	# weight initialization
	init = RandomNormal(stddev=0.02, seed = 20)
	# first layer convolutional layer
	g = Conv2D(n_filters, (3,3), padding='same', kernel_initializer=init)(input_layer)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# second convolutional layer
	g = Conv2D(n_filters, (3,3), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	# concatenate merge channel-wise with input layer
	g = Concatenate()([g, input_layer])
	return g
 
# define the standalone generator model
def define_generator(image_shape, n_resnet=7):
	# weight initialization
	init = RandomNormal(stddev=0.02, seed = 20)
	# image input
	in_image = Input(shape=image_shape)
	# c7s1-64
	g = Conv2D(32, (9,9), padding='same', kernel_initializer=init)(in_image)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# d128
	g = Conv2D(64, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# d256
	g = Conv2D(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# R256
	for _ in range(n_resnet):
		g = resnet_block(128, g)
	# u128
	g = Conv2DTranspose(64, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# u64
	g = Conv2DTranspose(32, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# c7s1-3
	g = Conv2D(3, (9,9), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	out_image = Activation('tanh')(g)
	# define model
	model = Model(in_image, out_image)
	return model
 
# define a composite model for updating generators by adversarial and cycle loss
def define_composite_model(g_model_1, d_model, g_model_2, image_shape):
	# ensure the model we're updating is trainable
	g_model_1.trainable = True
	# mark discriminator as not trainable
	d_model.trainable = False
	# mark other generator model as not trainable
	g_model_2.trainable = False
	# discriminator element
	input_gen = Input(shape=image_shape)
	gen1_out = g_model_1(input_gen)
	# forward cycle
	output_f = g_model_2(gen1_out)
	output_d = d_model(gen1_out)
	# identity element
	input_id = Input(shape=image_shape)
	# backward cycle
	gen2_out = g_model_2(input_id)
	output_b = g_model_1(gen2_out)
	output_id = g_model_1(input_id)
	# define model graph
	model = Model([input_gen, input_id], [output_d, output_id, output_f, output_b])
	# define optimization algorithm configuration
	opt = Adam(lr=0.000005, beta_1=0.5)
	# compile model with weighting of least squares loss and L1 loss
	model.compile(loss=['mse', 'mse', 'mse', 'mse'], loss_weights=[1, 5, 10, 10], optimizer=opt)
	return model
 
# load and prepare training images
def load_real_samples(filename):
	# load the dataset
	data = load(filename)
	# unpack arrays
	X1, X2 = data['arr_0'], data['arr_1']
	# scale from [0,255] to [-1,1]
	X1 = (X1 - 127.5) / 127.5
	X2 = (X2 - 127.5) / 127.5
	return [X1, X2]
 

# select a batch of random samples, returns images and target
def generate_real_samples(dataset, n_samples, patch_shape, i, bat_per_epo):
#	choose random instances
	i = i - bat_per_epo*math.floor(float(i/bat_per_epo))
#	ix = randint(0, dataset.shape[0], n_samples)
	# retrieve selected images
	ix = range(n_samples*i,n_samples*(i+1))
	X = dataset[ix]
	# generate 'real' class labels (1)
	y = ones((n_samples, patch_shape, patch_shape, 1))
	return X, y
 
# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, dataset, patch_shape):
	# generate fake instance
	X = g_model.predict(dataset)
	# create 'fake' class labels (0)
	y = zeros((len(X), patch_shape, patch_shape, 1))
	return X, y
 
# save the generator models to file
#def save_models(step, g_model_AtoB, g_model_BtoA):
def save_models(step, g_model_BtoA):
	# save the first generator model
#	filename1 = 'g_model_AtoB_%06d.h5' % (step+1)
#	g_model_AtoB.save(filename1)
	# save the second generator model
	filename2 = 'g_model_BtoA_%06d.h5' % (step+1)
	g_model_BtoA.save(filename2)
	print('>Saved: %s ' % (filename2))

 
# generate samples and save as a plot and save the model
def summarize_performance(step, data, OG, g_model, trainX, name, fid_min, n_samples=17):
	# select a sample of input images
	# unpack arrays
	X1 = data['arr_0']
	X2 = OG['arr_0']
	# scale from [0,255] to [-1,1]
	X1 = (X1 - 127.5) / 127.5
	X2 = (X2 - 127.5) / 127.5
	X_in = X1
	#X_in, _ = generate_real_samples(trainX, n_samples, 0)
	# generate translated images
	X_out, _ = generate_fake_samples(g_model, X_in, 0)
	# scale all pixels from [-1,1] to [0,1]
	X_in = (X_in + 1) / 2.0
	X2 = (X2 + 1) / 2.0
	X_out = (X_out + 1) / 2.0
	print(X2.shape, X_out.shape)
	fid = FID(X2,X_out)
# 	if fid < fid_min:
# 	# plot real images
# 		for j in range(7):
# 			for i in range(5):
# 				pyplot.subplot(2, 5, 1 + i)
# 				pyplot.axis('off')
# 				pyplot.imshow(X_in[5*j + i])
# 	# plot translated image
# 			for i in range(5):
# 				pyplot.subplot(2, 5, 1 + 5 + i)
# 				pyplot.axis('off')
# 				pyplot.imshow(X_out[5*j + i])
# 	# save plot to file
# 			filename1 = '%s_generated_plot_%06d_iter_%d.png' % (name, (step+1),j)
# 			pyplot.savefig(filename1)
# 			pyplot.close()          
	return fid
 
# update image pool for fake images
def update_image_pool(pool, images, max_size=50):
	selected = list()
	for image in images:
		if len(pool) < max_size:
			# stock the pool
			pool.append(image)
			selected.append(image)
		elif random() < 0.5:
			# use image, but don't add it to the pool
			selected.append(image)
		else:
			# replace an existing image and use replaced image
			ix = randint(0, len(pool))
			selected.append(pool[ix])
			pool[ix] = image
	return asarray(selected)
 
# train cyclegan models
def train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA, dataset):
	# define properties of the training run
	dif_df = pd.DataFrame(columns = ['Iteration', 'FID'])
	n_epochs, n_batch, = 500, 7
	# determine the output square shape of the discriminator
	n_patch = d_model_A.output_shape[1]
	# unpack dataset
	trainA, trainB = dataset
	#trainB = np.concatenate((trainB,trainB), axis = 0)
	#trainA = np.random.rand(*trainA.shape).argsort(axis=0)
	print(trainA.shape, trainB.shape)
	# prepare image pool for fakes
	poolA, poolB = list(), list()
	# calculate the number of batches per training epoch 119, 526
	bat_per_epo = int(len(trainA) / n_batch) #434/6 = 72
	# calculate the number of training iterations
	n_steps = int(bat_per_epo * n_epochs) #200000
	df = pd.DataFrame(columns = ['i','g_loss2','g_loss1','dA_loss1','dA_loss2','dB_loss1','dB_loss2','_1','_2','_3','_4','_5','_6','_7','_8'])
	data = load('/kaggle/input/mb-test/MB_Testing_all.npz')
	OG = load('/kaggle/input/mb-testing-og/Mb_blur_Testing_OG.npz')
	fid_min = 20
	print(f'Batch per epoch is {bat_per_epo} \nN_steps is {n_steps}')
	# manually enumerate epochs
	lrt = 0.002
	for i in range(n_steps):
		# select a batch of real samples
		X_realA, y_realA = generate_real_samples(trainA, n_batch, n_patch, i, bat_per_epo)
		X_realB, y_realB = generate_real_samples(trainB, n_batch, n_patch, i, bat_per_epo)
		# generate a batch of fake samples
		X_fakeA, y_fakeA = generate_fake_samples(g_model_BtoA, X_realB, n_patch)
		X_fakeB, y_fakeB = generate_fake_samples(g_model_AtoB, X_realA, n_patch)
		# update fakes from pool
		X_fakeA = update_image_pool(poolA, X_fakeA)
		X_fakeB = update_image_pool(poolB, X_fakeB)
		# update generator B->A via adversarial and cycle loss
		g_loss2, _1, _2, _3, _4  = c_model_BtoA.train_on_batch([X_realB, X_realA], [y_realA, X_realA, X_realB, X_realA])
# 		print(g_loss2,_1,_2,_3,_4)
# 		print(c_model_BtoA.metrics_names)['loss', 'model_2_loss', 'model_1_loss', 'model_loss', 'model_1_1_loss']
# 		print(d_model_B.metrics_names) ['loss']
# 		print(g_model_AtoB.metrics_names) []
		# update discriminator for A -> [real/fake]
		dA_loss1 = d_model_A.train_on_batch(X_realA, y_realA)
		dA_loss2 = d_model_A.train_on_batch(X_fakeA, y_fakeA)
		# update generator A->B via adversarial and cycle loss
		g_loss1, _5, _6, _7, _8 = c_model_AtoB.train_on_batch([X_realA, X_realB], [y_realB, X_realB, X_realA, X_realB])
		# update discriminator for B -> [real/fake]
		dB_loss1 = d_model_B.train_on_batch(X_realB, y_realB)
		dB_loss2 = d_model_B.train_on_batch(X_fakeB, y_fakeB)
		df.loc[len(df.index)] = [i,g_loss2,g_loss1,dA_loss1,dA_loss2,dB_loss1,dB_loss2,_1,_2,_3,_4,_5,_6,_7,_8]
		# summarize performance
		print('>%d, dA[%.3f,%.3f] dB[%.3f,%.3f] g[%.3f,%.3f]' % (i+1, dA_loss1,dA_loss2, dB_loss1,dB_loss2, g_loss1,g_loss2))
		# evaluate the model performance every so often
		if 4*((i)) % (bat_per_epo) == 0:
			fid_o = summarize_performance(i, data, OG, g_model_BtoA, trainB, 'BtoA', fid_min)
			print(fid_o, fid_min)
			df.to_csv('Losses.csv')
			#summarize_performance(i,data, OG, g_model_BtoA, trainB, 'BtoA')
			dif_df.loc[len(dif_df)] = [i,fid_o]
			if fid_o < fid_min:
				fid_min = fid_o
				save_models(i,g_model_BtoA)
				filename2 = 'g_model_BtoA_%06d.h5' % (i+1)
				g_model_AtoB.save('g_model_AtoB.h5')     
				d_model_A.save('d_model_A.h5')
				d_model_B.save('d_model_B.h5')
				c_model_AtoB.save('c_model_AtoB.h5')
				c_model_BtoA.save('c_model_BtoA.h5')
			a = 'DIF' + str(i) + '.csv'
			dif_df.to_csv('DIF.csv')

dataset = load_real_samples('/kaggle/input/mb-training-full-data1/MB_Blur_patches_all_files_May1.npz')
print('Loaded', dataset[0].shape, dataset[1].shape)
# define input shape based on the loaded dataset
image_shape = dataset[0].shape[1:]
# generator: A -> B
g_model_AtoB = define_generator(image_shape)
# generator: B -> A
g_model_BtoA = define_generator(image_shape)
# discriminator: A -> [real/fake]
d_model_A = define_discriminator(image_shape, lrt)
# discriminator: B -> [real/fake]
d_model_B = define_discriminator(image_shape, lrt)
# composite: A -> B -> [real/fake, A]
c_model_AtoB = define_composite_model(g_model_AtoB, d_model_B, g_model_BtoA, image_shape, lrt)
# composite: B -> A -> [real/fake, B]
c_model_BtoA = define_composite_model(g_model_BtoA, d_model_A, g_model_AtoB, image_shape, lrt)
# g_model_AtoB = load_model('/kaggle/input/most-recent-models-21-march/g_model_AtoB.h5',custom_objects={'InstanceNormalization':InstanceNormalization})
# g_model_BtoA = load_model('/kaggle/input/most-recent-models-21-march/c_model_BtoA.h5',custom_objects={'InstanceNormalization':InstanceNormalization})
# d_model_A = load_model('/kaggle/input/most-recent-models-21-march/d_model_A.h5',custom_objects={'InstanceNormalization':InstanceNormalization})
# d_model_B = load_model('/kaggle/input/most-recent-models-21-march/d_model_B.h5',custom_objects={'InstanceNormalization':InstanceNormalization})
# c_model_AtoB = load_model('/kaggle/input/c-atob-march21/c_model_AtoB.h5',custom_objects={'InstanceNormalization':InstanceNormalization})
# c_model_BtoA = load_model('/kaggle/input/most-recent-models-21-march/c_model_BtoA.h5',custom_objects={'InstanceNormalization':InstanceNormalization})
# train models
train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA, dataset)